# Step 0. Setup and Paths initialized.

In [1]:
# Two Race QA - Comparing performances across races.
# Goal - Loading 2 sessions and answering practical business style questions.
# Skills - Data Loading, Filtering, Joining, Grouping, Aggregation, Window Ops, Merge_asof, Vectorization, Defensiveness. 

from pathlib import Path
import pandas as pd
import numpy as np
import json, glob

PROJECT_DIR = Path.cwd().parents[0]
PROCESSED_DATA_DIR = PROJECT_DIR / "data" / "processed"

print(f"Project Directory: {PROJECT_DIR}")
print(f"Processed Data Directory: {PROCESSED_DATA_DIR}")

Project Directory: /Users/pratyushagarwal/Desktop/PROJECTS/F1_Projects/race_weekend_data_pipeline
Processed Data Directory: /Users/pratyushagarwal/Desktop/PROJECTS/F1_Projects/race_weekend_data_pipeline/data/processed


# Step 1. Picking 2 race sessions.

In [2]:
# We will pick 2 race sessions (endswith _R). Can be hardcoded or picking the first 2 automatically.

all_sessions = sorted(Path(p).name for p in glob.glob(str(PROCESSED_DATA_DIR / "*")) if Path(p).is_dir())
race_sessions = [r for r in all_sessions if r.endswith('_R')]

race_sessions[:10], len(race_sessions)

(['2024_01_R',
  '2024_02_R',
  '2024_03_R',
  '2024_04_R',
  '2024_05_R',
  '2024_06_R',
  '2024_07_R',
  '2024_08_R',
  '2024_09_R',
  '2024_10_R'],
 24)

In [3]:
# Choosing 2 races to compare
s1 = race_sessions[5] if len(race_sessions) > 5 else (race_sessions[0] if len(race_sessions) > 0 else None)
s2 = race_sessions[15] if len(race_sessions) > 15 else (race_sessions[1] if len(race_sessions) > 1 else None)

s1, s2

('2024_06_R', '2024_16_R')

# Step 2. Loading processed tables for both sessions.

In [4]:
# Loading fact tables for s1 and s2 - suffix columns to keep them distinct when necessary.

def load_session_tables(session_key: str):
    d = PROCESSED_DATA_DIR / session_key
    paths = {p.stem: p for p in d.glob("*.parquet")}
    fl = pd.read_parquet(paths['fact_laps']) if 'fact_laps' in paths else pd.DataFrame()
    fp = pd.read_parquet(paths['fact_pitstops']) if 'fact_pitstops' in paths else pd.DataFrame()
    fw = pd.read_parquet(paths['fact_weather']) if 'fact_weather' in paths else pd.DataFrame()
    dd = pd.read_parquet(paths['dim_drivers']) if 'dim_drivers' in paths else pd.DataFrame()

    return fl, fp, fw, dd

In [5]:
fl1, fp1, fw1, dd1 = load_session_tables(s1)
fl2, fp2, fw2, dd2 = load_session_tables(s2)

(fl1.shape, fp1.shape, fw1.shape, dd1.shape), (fl2.shape, fp2.shape, fw2.shape, dd2.shape)

(((1111, 16), (0, 5), (150, 10), (20, 6)),
 ((1008, 16), (0, 5), (133, 10), (20, 6)))

# Step 3. Normalizing driver keys and enriching with names.

In [6]:
# Normalizing driver_number to string and attaching names / teams for readability.

def enrich_laps(fl: pd.DataFrame, dd: pd.DataFrame) -> pd.DataFrame:
    if fl.empty:
        return fl
    
    fl = fl.copy()
    fl['driver_number'] = fl['driver_number'].astype(str)
    if not dd.empty:
        dd = dd.copy()
        dd['driver_number'] = dd['driver_number'].astype(str)
        fl = fl.merge(
            dd[['session_key', 'driver_number', 'full_name', 'team']],
            on=['session_key', 'driver_number'],
            how='left'
        )
    
    return fl

In [7]:
fl1e = enrich_laps(fl1, dd1)
fl2e = enrich_laps(fl2, dd2)

display(fl1e.drop_duplicates().head(2).style.set_caption(f"Enriched Laps - Session: {s1}"))
display(fl2e.drop_duplicates().head(2).style.set_caption(f"Enriched Laps - Session: {s2}"))

,driver_number,lap_number,tyre_compound,stint,track_status,tyre_life,is_accurate,speed_trap_kph,lap_time_ms,sector1_time_ms,sector2_time_ms,sector3_time_ms,is_inlap,is_outlap,is_pit,session_key,full_name,team
0,1,1.000000,MEDIUM,1,1,1.000000,False,305.000000,94338.000000,nan,34990.000000,26211.000000,False,False,False,2024_06_R,Max Verstappen,Red Bull Racing
1,1,2.000000,MEDIUM,1,1,2.000000,True,303.000000,93093.000000,31455.000000,35489.000000,26149.000000,False,False,False,2024_06_R,Max Verstappen,Red Bull Racing


,driver_number,lap_number,tyre_compound,stint,track_status,tyre_life,is_accurate,speed_trap_kph,lap_time_ms,sector1_time_ms,sector2_time_ms,sector3_time_ms,is_inlap,is_outlap,is_pit,session_key,full_name,team
0,1,1.000000,HARD,1,1,1.000000,False,298.000000,90354.000000,nan,31144.000000,28856.000000,False,False,False,2024_16_R,Max Verstappen,Red Bull Racing
1,1,2.000000,HARD,1,1,2.000000,True,316.000000,86170.000000,28048.000000,29474.000000,28648.000000,False,False,False,2024_16_R,Max Verstappen,Red Bull Racing


# Q1) Which drivers improved or regressed average race pace from Race A to Race B.

In [ ]:
# Question - Who got faster? Comparing average lap_time_ms across 2 races.

def pace_summary(fl: pd.DataFrame) -> pd.DataFrame:
    if fl.empty:
        return pd.DataFrame()
    
    df = fl.loc[~fl['is_pit'].fillna(False)].copy()
    return (
        df.groupby(['driver_number', 'full_name', 'team'], dropna=False)['lap_time_ms']
        .agg(
            laps='size',
            avg_ms='mean',
            std_ms='std'
        )
        .reset_index()
    )

p1 = pace_summary(fl1e).rename(columns={
    'avg_ms' : 'avg_ms_s1',
    'std_ms' : 'std_ms_s1',
    'laps' : 'laps_s1'
})
p2 = pace_summary(fl2e).rename(columns={
    'avg_ms' : 'avg_ms_s2', 
    'std_ms' : 'std_ms_s2',
    'laps' : 'laps_s2'
})

pace_delta = (
    p1.merge(
        p2,
        on=['driver_number', 'full_name', 'team'],
        how='inner'
    )
    .assign(delta_ms = lambda x: x['avg_ms_s2'] - x['avg_ms_s1'])
    .sort_values('delta_ms')
)

display(pace_delta.head(20))

# Interpretation Tip - Negative delta_ms means improvement. Faster in Race B compared to Race A.

,driver_number,full_name,team,laps_s1,avg_ms_s1,std_ms_s1,laps_s2,avg_ms_s2,std_ms_s2,delta_ms
16,63,George Russell,Mercedes,55,95984.672727,8724.530298,49,84137.306122,1584.030292,-11847.366605
14,44,Lewis Hamilton,Mercedes,55,95662.690909,9263.169130,49,84015.163265,1432.604335,-11647.527644
15,55,Carlos Sainz,Ferrari,55,95402.181818,11007.655661,51,84336.254902,1054.830031,-11065.926916
18,81,Oscar Piastri,McLaren,53,94646.000000,8603.088866,49,83625.285714,1107.343142,-11020.714286
3,14,Fernando Alonso,Aston Martin,55,95913.945455,8165.690092,49,84896.673469,1500.546489,-11017.271985
8,23,Alexander Albon,Williams,53,96318.075472,8029.355877,51,85308.117647,1217.429696,-11009.957825
1,10,Pierre Gasly,Alpine,55,96138.254545,8113.441728,48,85522.875000,1639.517737,-10615.379545
4,16,Charles Leclerc,Ferrari,55,94501.462963,8780.786533,51,84032.294118,859.673875,-10469.168845
11,3,Daniel Ricciardo,RB,55,95451.618182,5916.931426,51,85392.725490,1257.290294,-10058.892692
12,31,Esteban Ocon,Alpine,55,95915.218182,8361.159200,50,85958.380000,1593.116864,-9956.838182


# Q2) Who had better consistency (lower dispersion) race-over-race?

In [11]:
def consistency(fl):
    if fl.empty:
        return pd.DataFrame()
    
    d = (
        fl.loc[~fl['is_pit'].fillna(False)]
        .groupby(['driver_number', 'full_name'], dropna=False)['lap_time_ms']
        .agg(
            std_ms='std',
            laps='size'
        )
        .reset_index()
    )
    return d

c1 = consistency(fl1e).rename(columns={
    'std_ms' : 'std_ms_s1',
    'laps' : 'laps_s1'
})

c2 = consistency(fl2e).rename(columns={
    'std_ms' : 'std_ms_s2', 
    'laps' : 'laps_s2'
})

consistency_delta = (
    c1.merge(
        c2,
        on=['driver_number', 'full_name'],
        how='inner',   
    )
    .assign(
        delta_std = lambda x: x['std_ms_s2'] - x['std_ms_s1']
    )
    .sort_values('delta_std')
)

display(consistency_delta.head(20))

# Negative delta_std means more consistent in Race B vs Race A.

,driver_number,full_name,std_ms_s1,laps_s1,std_ms_s2,laps_s2,delta_std
15,55,Carlos Sainz,11007.655661,55,1054.830031,51,-9952.825630
4,16,Charles Leclerc,8780.786533,55,859.673875,51,-7921.112658
14,44,Lewis Hamilton,9263.169130,55,1432.604335,49,-7830.564795
18,81,Oscar Piastri,8603.088866,53,1107.343142,49,-7495.745725
2,11,Sergio Perez,8788.539078,53,1436.516528,49,-7352.022550
0,1,Max Verstappen,8631.601092,55,1422.337163,49,-7209.263929
16,63,George Russell,8724.530298,55,1584.030292,49,-7140.500006
8,23,Alexander Albon,8029.355877,53,1217.429696,51,-6811.926181
12,31,Esteban Ocon,8361.159200,55,1593.116864,50,-6768.042336
3,14,Fernando Alonso,8165.690092,55,1500.546489,49,-6665.143603


# Q3) Did teams improve? (Team-level averaging)

In [12]:
def team_pace(fl):
    if fl.empty:
        return pd.DataFrame()
    
    d = (
        fl.loc[~fl['is_pit'].fillna(False)]
        .groupby('team', dropna=False)['lap_time_ms']
        .mean()
        .rename('avg_ms')
        .reset_index()
    )
    return d

t1 = team_pace(fl1e).rename(columns={
    'avg_ms' : 'avg_ms_s1'
})

t2 = team_pace(fl2e).rename(columns={
    'avg_ms' : 'avg_ms_s2'
})

team_pace_delta = (
    t1.merge(
        t2,
        on='team',
        how='inner'
    )
    .assign(
        delta_ms = lambda x: x['avg_ms_s2'] - x['avg_ms_s1']
    )
    .sort_values('delta_ms')
)

display(team_pace_delta.head(20))

,team,avg_ms_s1,avg_ms_s2,delta_ms
6,Mercedes,95823.681818,84076.234694,-11747.447124
2,Ferrari,94955.954128,84184.274510,-10771.679619
9,Williams,95899.461538,85443.205882,-10456.255656
0,Alpine,96026.736364,85745.071429,-10281.664935
1,Aston Martin,95426.935185,85188.536842,-10238.398343
5,McLaren,93806.122642,83671.173469,-10134.949172
8,Red Bull Racing,94367.439252,84448.816327,-9918.622926
7,RB,95272.972727,85813.666667,-9459.306061
3,Haas F1 Team,94352.875000,85205.010101,-9147.864899
4,Kick Sauber,95280.250000,86311.580000,-8968.670000


# Q4) Did weather correlate with pace changes between races?

In [14]:
def weather_summary(fw):
    if fw.empty: 
        return pd.DataFrame()
    cols = [c for c in ['air_temp_c', 'track_temp_c', 'humidity_pct', 'wind_speed_mps'] if c in fw.columns]
    if not cols:
        return pd.DataFrame()
    return fw[cols].mean().to_frame('avg').T

ws1 = weather_summary(fw1)
ws2 = weather_summary(fw2)

display(ws1.head(20))
display(ws2.head(20))

# If Race B had higher track temps and pace delta is positive (slower), then that's a plausible contributing factor.

,air_temp_c,track_temp_c,humidity_pct,wind_speed_mps
avg,28.522,44.664,59.006667,0.855741


,air_temp_c,track_temp_c,humidity_pct,wind_speed_mps
avg,33.101504,49.311278,32.285714,0.421888


# Q5) Who faded vs who surged across the race? 

In [16]:
def split_halves(fl):
    if fl.empty:
        return pd.DataFrame()
    
    df = fl.loc[~fl['is_pit'].fillna(False)].copy()
    # driver-wise halfway lap.
    half = df.groupby('driver_number', dropna=False)['lap_number'].transform('max') // 2
    df['half'] = np.where(df['lap_number'] <= half, 'H1', 'H2')

    d = (
        df.groupby(['driver_number', 'full_name', 'half'], dropna=False)['lap_time_ms']
        .median()
        .unstack('half')
    )

    if 'H1' in d.columns and 'H2' in d.columns:
        d['delta_H2_minus_H1'] = d['H2'] - d['H1']
    return d.reset_index()

h1 = split_halves(fl1e).rename(columns={'delta_H2_minus_H1' : 'delta_halves_s1'})
h2 = split_halves(fl2e).rename(columns={'delta_H2_minus_H1' : 'delta_halves_s2'})

halves_compare = (
    h1.merge(
        h2,
        on=['driver_number', 'full_name'],
        how='inner'
    )
)

halves_compare.head(20)

# Interpretation tip: Positive delta → slower second half; negative → stronger finish.

half,driver_number,full_name,H1_x,H2_x,delta_halves_s1,H1_y,H2_y,delta_halves_s2
0,1,Max Verstappen,92923.0,91480.0,-1443.0,85231.0,83452.0,-1779.0
1,10,Pierre Gasly,93976.0,92786.0,-1190.0,85851.0,84598.0,-1253.0
2,11,Sergio Perez,93198.0,91569.0,-1629.0,85439.0,83611.0,-1828.0
3,14,Fernando Alonso,94219.0,92482.0,-1737.0,85684.5,83703.0,-1981.5
4,16,Charles Leclerc,93118.0,91576.5,-1541.5,84433.0,83555.0,-878.0
5,18,Lance Stroll,93861.0,92877.5,-983.5,86083.5,84527.5,-1556.0
6,20,Kevin Magnussen,94221.0,92352.0,-1869.0,85812.0,84408.0,-1404.0
7,22,Yuki Tsunoda,93936.0,92024.0,-1912.0,87538.0,88473.0,935.0
8,23,Alexander Albon,94077.5,92947.0,-1130.5,85722.5,84678.0,-1044.5
9,24,Guanyu Zhou,94318.0,92944.5,-1373.5,86177.0,85759.5,-417.5


# Save a two-race summary JSON

In [17]:
summary = {
    'session_A' : s1,
    'session_B' : s2,
    'top_improvers_by_avg_pace' : pace_delta[["driver_number","full_name","team","avg_ms_s1","avg_ms_s2","delta_ms"]].head(10).to_dict("records"),
    'halves_comparison' : halves_compare.head(20).to_dict("records"),
    'consistency_comparison' : consistency_delta.head(20).to_dict("records"),
    'team_pace_comparison' : team_pace_delta.head(20).to_dict("records"),
    'weather_summary_session_A' : ws1.to_dict("records")
}

out_path = PROJECT_DIR / "notebooks" / "03_two_race_QA_summary.json"
out_path.write_text(json.dumps(summary, indent=2))
out_path

PosixPath('/Users/pratyushagarwal/Desktop/PROJECTS/F1_Projects/race_weekend_data_pipeline/notebooks/03_two_race_QA_summary.json')